In [31]:
import pandas as pd
import numpy as np
%config Completer.use_jedi=False

## Spliting an object into groups
---

In [32]:
df = pd.DataFrame(
        [
            ("bird", "Falconiformes", 389.0),
            ("bird", "Psittaciformes", 24.0),
            ("mammal", "Carnivora", 80.2),
            ("mammal", "Primates", np.nan),
            ("mammal", "Carnivora", 58),
        ],
        index=["falcon", "parrot", "lion", "monkey", "leopard"],
        columns=("class", "order", "max_speed"),
    )
df

,class,order,max_speed
falcon,bird,Falconiformes,389.0
parrot,bird,Psittaciformes,24.0
lion,mammal,Carnivora,80.2
monkey,mammal,Primates,NaN
leopard,mammal,Carnivora,58.0


In [33]:
grouped = df.groupby('class')
grouped = df.groupby("order", axis="columns")
grouped = df.groupby(["class", "order"])

In [34]:
#  MultiIndex on columns
df = pd.DataFrame(
        {
            "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
            "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
            "C": np.random.randn(8),
            "D": np.random.randn(8),
        }
    )
df2 = df.set_index(["A", "B"])
print(df2)
grouped = df2.groupby(level=df2.index.names.difference(["B"]))
grouped.sum()

                  C         D
A   B                        
foo one    0.510888  0.345374
bar one   -0.471014  0.532227
foo two   -1.051775 -0.378660
bar three -1.880993 -1.453361
foo two   -0.047773  0.922374
bar two    2.324554 -0.344190
foo one   -0.137977 -0.182738
    three -0.473664  0.592003


,C,D
A,,
bar,-0.027454,-1.265324
foo,-1.200301,1.298354


In [35]:
# duplicated value in index
lst = [1, 2, 3, 1, 2, 3]
s = pd.Series([1,2,3,10,20,30], lst)
s

1     1
2     2
3     3
1    10
2    20
3    30
dtype: int64

In [36]:
grouped = s.groupby(level=0)
grouped.first()

1    1
2    2
3    3
dtype: int64

In [37]:
grouped.last()

1    10
2    20
3    30
dtype: int64

In [38]:
grouped.sum()

1    11
2    22
3    33
dtype: int64

## GroupBy sorting
---

In [39]:
df2 = pd.DataFrame({"X": ["B", "B", "A", "A"], "Y": [1, 2, 3, 4]})
df2

,X,Y
0,B,1
1,B,2
2,A,3
3,A,4


In [40]:
df2.groupby(['X']).sum()

,Y
X,
A,7
B,3


In [41]:
df2.groupby(['X'], sort=False).sum()

,Y
X,
B,3
A,7


## Groupby dropna
---

In [42]:
df_list = [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]
df_dropna = pd.DataFrame(df_list, columns=["a", "b", "c"])
df_dropna

,a,b,c
0,1,2.0,3
1,1,NaN,4
2,2,1.0,3
3,1,2.0,2


In [43]:
df_dropna.groupby(by=["b"], dropna=True).sum()

,a,c
b,,
1.0,2,3
2.0,2,5


In [44]:
df_dropna.groupby(by=["b"], dropna=False).sum()

,a,c
b,,
1.0,2,3
2.0,2,5
NaN,1,4


## GroupBy object attributes
---

In [45]:
df

,A,B,C,D
0,foo,one,0.510888,0.345374
1,bar,one,-0.471014,0.532227
2,foo,two,-1.051775,-0.378660
3,bar,three,-1.880993,-1.453361
4,foo,two,-0.047773,0.922374
5,bar,two,2.324554,-0.344190
6,foo,one,-0.137977,-0.182738
7,foo,three,-0.473664,0.592003


In [46]:
df.groupby('A')

In [47]:
df.groupby('A').groups

{'bar': [1, 3, 5], 'foo': [0, 2, 4, 6, 7]}

In [48]:
grouped = df.groupby(["A", "B"])
grouped.groups

{('bar', 'one'): [1], ('bar', 'three'): [3], ('bar', 'two'): [5], ('foo', 'one'): [0, 6], ('foo', 'three'): [7], ('foo', 'two'): [2, 4]}

In [49]:
public_props = (name for name in dir(grouped) if not name.startswith('_'))
for name in public_props:
    print(name)

A
B
C
D
agg
aggregate
all
any
apply
backfill
bfill
boxplot
corr
corrwith
count
cov
cumcount
cummax
cummin
cumprod
cumsum
describe
diff
dtypes
expanding
ffill
fillna
filter
first
get_group
groups
head
hist
idxmax
idxmin
indices
last
mad
max
mean
median
min
ndim
ngroup
ngroups
nth
nunique
ohlc
pad
pct_change
pipe
plot
prod
quantile
rank
resample
rolling
sample
sem
shift
size
skew
std
sum
tail
take
transform
tshift
var


## GroupBy with MultiIndex
---

In [50]:
arrays = [
        ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
        ["one", "two", "one", "two", "one", "two", "one", "two"],
    ]
index = pd.MultiIndex.from_arrays(arrays, names=["first", "second"])
s = pd.Series(np.random.randn(8), index=index)
s

first  second
bar    one       0.212358
       two      -0.074990
baz    one       0.355724
       two      -0.664744
foo    one       1.407274
       two      -0.339245
qux    one      -1.568455
       two       0.137711
dtype: float64

In [51]:
grouped = s.groupby(level=0)
grouped.sum()

first
bar    0.137368
baz   -0.309020
foo    1.068029
qux   -1.430744
dtype: float64

In [52]:
s.groupby(level="second").sum()

second
one    0.406900
two   -0.941267
dtype: float64

In [53]:
s.sum(level="second")

second
one    0.406900
two   -0.941267
dtype: float64

In [54]:
s.groupby(level=["first", "second"]).sum()

first  second
bar    one       0.212358
       two      -0.074990
baz    one       0.355724
       two      -0.664744
foo    one       1.407274
       two      -0.339245
qux    one      -1.568455
       two       0.137711
dtype: float64

In [55]:
s.groupby(["first", "second"]).sum()

first  second
bar    one       0.212358
       two      -0.074990
baz    one       0.355724
       two      -0.664744
foo    one       1.407274
       two      -0.339245
qux    one      -1.568455
       two       0.137711
dtype: float64

## Grouping DataFrame with Index levels and columns
---

In [56]:
arrays = [
        ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
        ["one", "two", "one", "two", "one", "two", "one", "two"],
    ]
index = pd.MultiIndex.from_arrays(arrays, names=["first", "second"])

df = pd.DataFrame({"A": [1, 1, 1, 1, 2, 2, 3, 3], "B": np.arange(8)}, index=index)
df

A  B
first second      
bar   one     1  0
      two     1  1
baz   one     1  2
      two     1  3
foo   one     2  4
      two     2  5
qux   one     3  6
      two     3  7

In [57]:
df.groupby([pd.Grouper(level=1), 'A']).sum()

B
second A   
one    1  2
       2  4
       3  6
two    1  4
       2  5
       3  7

In [58]:
df.groupby([pd.Grouper(level="second"), "A"]).sum()

B
second A   
one    1  2
       2  4
       3  6
two    1  4
       2  5
       3  7

In [59]:
df.groupby(["second", "A"]).sum()

B
second A   
one    1  2
       2  4
       3  6
two    1  4
       2  5
       3  7

## DataFrame column selection in GroupBy
---

In [60]:
# grouped = df.groupby(["A"])
# grouped_C = grouped["C"]
# grouped_D = grouped["D"]


## Iterating through groups
---

In [61]:
df = pd.DataFrame(
        {
            "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
            "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
            "C": np.random.randn(8),
            "D": np.random.randn(8),
        }
    )
df

,A,B,C,D
0,foo,one,-1.610649,0.088701
1,bar,one,1.459099,-0.699461
2,foo,two,-0.377928,0.578508
3,bar,three,-0.324777,0.791631
4,foo,two,0.375390,0.707167
5,bar,two,0.649998,-1.355308
6,foo,one,-0.777828,-1.571484
7,foo,three,0.508563,-1.057590


In [62]:
for name, group in grouped:
    print(name)
    print(group)

bar
first  second
bar    one       0.212358
       two      -0.074990
dtype: float64
baz
first  second
baz    one       0.355724
       two      -0.664744
dtype: float64
foo
first  second
foo    one       1.407274
       two      -0.339245
dtype: float64
qux
first  second
qux    one      -1.568455
       two       0.137711
dtype: float64


In [63]:
for name, group in df.groupby(['A', 'B']):
    print(name)
    print(group)

('bar', 'one')
     A    B         C         D
1  bar  one  1.459099 -0.699461
('bar', 'three')
     A      B         C         D
3  bar  three -0.324777  0.791631
('bar', 'two')
     A    B         C         D
5  bar  two  0.649998 -1.355308
('foo', 'one')
     A    B         C         D
0  foo  one -1.610649  0.088701
6  foo  one -0.777828 -1.571484
('foo', 'three')
     A      B         C        D
7  foo  three  0.508563 -1.05759
('foo', 'two')
     A    B         C         D
2  foo  two -0.377928  0.578508
4  foo  two  0.375390  0.707167


## Selecting a group
---

In [64]:
grouped.get_group("bar")

first  second
bar    one       0.212358
       two      -0.074990
dtype: float64

In [65]:
df.groupby(["A", "B"]).get_group(("bar", "one"))

,A,B,C,D
1,bar,one,1.459099,-0.699461


## Aggregation
---

In [66]:
grouped = df.groupby("A")
grouped.aggregate(np.sum)

,C,D
A,,
bar,1.784320,-1.263138
foo,-1.882452,-1.254698


In [67]:
grouped = df.groupby(["A", "B"])
grouped.aggregate(np.sum)

C         D
A   B                        
bar one    1.459099 -0.699461
    three -0.324777  0.791631
    two    0.649998 -1.355308
foo one   -2.388476 -1.482783
    three  0.508563 -1.057590
    two   -0.002538  1.285676

In [68]:
grouped = df.groupby(["A", "B"], as_index=False)
grouped.aggregate(np.sum)

,A,B,C,D
0,bar,one,1.459099,-0.699461
1,bar,three,-0.324777,0.791631
2,bar,two,0.649998,-1.355308
3,foo,one,-2.388476,-1.482783
4,foo,three,0.508563,-1.057590
5,foo,two,-0.002538,1.285676


In [69]:
df.groupby("A", as_index=False).sum()

,A,C,D
0,bar,1.784320,-1.263138
1,foo,-1.882452,-1.254698


In [70]:
df.groupby(["A", "B"]).sum().reset_index()

,A,B,C,D
0,bar,one,1.459099,-0.699461
1,bar,three,-0.324777,0.791631
2,bar,two,0.649998,-1.355308
3,foo,one,-2.388476,-1.482783
4,foo,three,0.508563,-1.057590
5,foo,two,-0.002538,1.285676


In [71]:
grouped.size()

,A,B,size
0,bar,one,1
1,bar,three,1
2,bar,two,1
3,foo,one,2
4,foo,three,1
5,foo,two,2


In [72]:
grouped.describe()

C                                                                        \
  count      mean       std       min       25%       50%       75%       max   
0   1.0  1.459099       NaN  1.459099  1.459099  1.459099  1.459099  1.459099   
1   1.0 -0.324777       NaN -0.324777 -0.324777 -0.324777 -0.324777 -0.324777   
2   1.0  0.649998       NaN  0.649998  0.649998  0.649998  0.649998  0.649998   
3   2.0 -1.194238  0.588894 -1.610649 -1.402444 -1.194238 -0.986033 -0.777828   
4   1.0  0.508563       NaN  0.508563  0.508563  0.508563  0.508563  0.508563   
5   2.0 -0.001269  0.532676 -0.377928 -0.189599 -0.001269  0.187060  0.375390   

      D                                                                        
  count      mean       std       min       25%       50%       75%       max  
0   1.0 -0.699461       NaN -0.699461 -0.699461 -0.699461 -0.699461 -0.699461  
1   1.0  0.791631       NaN  0.791631  0.791631  0.791631  0.791631  0.791631  
2   1.0 -1.355308       NaN -1.355308 -1.355308 -1.355308 -1.355308 -1.355308  
3   2.0 -0.741391  1.173928 -1.571484 -1.156438 -0.741391 -0.326345  0.088701  
4   1.0 -1.057590       NaN -1.057590 -1.057590 -1.057590 -1.057590 -1.057590  
5   2.0  0.642838  0.090976  0.578508  0.610673  0.642838  0.675003  0.707167

In [73]:
ll = [['foo', 1], ['foo', 2], ['foo', 2], ['bar', 1], ['bar', 1]]
df4 = pd.DataFrame(ll, columns=['A', 'B'])
df4

,A,B
0,foo,1
1,foo,2
2,foo,2
3,bar,1
4,bar,1


In [74]:
df4.groupby('A')['B'].nunique()

A
bar    1
foo    2
Name: B, dtype: int64

## Applying multiple functions at once
---

In [75]:
grouped = df.groupby("A")
grouped["C"].agg([np.sum, np.mean, np.std])

,sum,mean,std
A,,,
bar,1.784320,0.594773,0.893219
foo,-1.882452,-0.376490,0.870756


In [76]:
grouped.agg([np.sum, np.mean, np.std])

C                             D                    
          sum      mean       std       sum      mean       std
A                                                              
bar  1.784320  0.594773  0.893219 -1.263138 -0.421046  1.100215
foo -1.882452 -0.376490  0.870756 -1.254698 -0.250940  1.014376

In [77]:
(grouped["C"].agg([np.sum, np.mean, np.std]).rename(columns={"sum": "foo", "mean": "bar", "std": "baz"}))

,foo,bar,baz
A,,,
bar,1.784320,0.594773,0.893219
foo,-1.882452,-0.376490,0.870756


## Named aggregation
---

In [79]:
animals = pd.DataFrame(
        {
            "kind": ["cat", "dog", "cat", "dog"],
            "height": [9.1, 6.0, 9.5, 34.0],
            "weight": [7.9, 7.5, 9.9, 198.0],
        }
    )
animals

,kind,height,weight
0,cat,9.1,7.9
1,dog,6.0,7.5
2,cat,9.5,9.9
3,dog,34.0,198.0


In [80]:
 animals.groupby("kind").agg(
   ....:     min_height=pd.NamedAgg(column="height", aggfunc="min"),
   ....:     max_height=pd.NamedAgg(column="height", aggfunc="max"),
   ....:     average_weight=pd.NamedAgg(column="weight", aggfunc=np.mean),
   ....: )

,min_height,max_height,average_weight
kind,,,
cat,9.1,9.5,8.90
dog,6.0,34.0,102.75


In [85]:
animals.groupby("kind").agg(
        min_height=("height", "min"),
        max_height=("height", "max"),
        average_weight=("weight", np.mean),
        total_weight=('weight', "sum")
    )

,min_height,max_height,average_weight,total_weight
kind,,,,
cat,9.1,9.5,8.90,17.8
dog,6.0,34.0,102.75,205.5


In [82]:
animals.groupby("kind").agg(
        **{
            "total weight": pd.NamedAgg(column="weight", aggfunc=sum)
        }
    )


,total weight
kind,
cat,17.8
dog,205.5


## Applying different functions to DataFrame columns
---

In [86]:
grouped.agg({"C": np.sum, "D": lambda x: np.std(x, ddof=1)})

,C,D
A,,
bar,1.784320,1.100215
foo,-1.882452,1.014376


In [87]:
grouped.agg({"C": "sum", "D": "std"})

,C,D
A,,
bar,1.784320,1.100215
foo,-1.882452,1.014376


## Cython-optimized aggregation functions
---

In [88]:
df.groupby("A").sum()

,C,D
A,,
bar,1.784320,-1.263138
foo,-1.882452,-1.254698


In [89]:
df.groupby(["A", "B"]).mean()

C         D
A   B                        
bar one    1.459099 -0.699461
    three -0.324777  0.791631
    two    0.649998 -1.355308
foo one   -1.194238 -0.741391
    three  0.508563 -1.057590
    two   -0.001269  0.642838

## Transformation
---

In [90]:
index = pd.date_range("10/1/1999", periods=1100)
ts = pd.Series(np.random.normal(0.5, 2, 1100), index)
ts

1999-10-01   -3.065981
1999-10-02    0.631078
1999-10-03   -1.885736
1999-10-04    4.064267
1999-10-05    1.452018
                ...   
2002-09-30   -0.009068
2002-10-01   -2.196776
2002-10-02   -0.529350
2002-10-03    0.730341
2002-10-04    4.044136
Freq: D, Length: 1100, dtype: float64

In [91]:
ts = ts.rolling(window=100, min_periods=100).mean().dropna()

In [92]:
ts.head()

2000-01-08    0.360226
2000-01-09    0.349928
2000-01-10    0.368669
2000-01-11    0.413724
2000-01-12    0.384109
Freq: D, dtype: float64

In [93]:
ts.tail()

2002-09-30    0.657819
2002-10-01    0.635248
2002-10-02    0.595145
2002-10-03    0.573520
2002-10-04    0.605681
Freq: D, dtype: float64

## Filtration
---

In [96]:
sf = pd.Series([1, 1, 2, 3, 3, 3])
print(sf)
sf.groupby(sf).filter(lambda x: x.sum() > 2)

0    1
1    1
2    2
3    3
4    3
5    3
dtype: int64


3    3
4    3
5    3
dtype: int64

In [97]:
dff = pd.DataFrame({"A": np.arange(8), "B": list("aabbbbcc")})
dff

,A,B
0,0,a
1,1,a
2,2,b
3,3,b
4,4,b
5,5,b
6,6,c
7,7,c


In [100]:
dff.groupby('B').filter(lambda x : len(x) > 2)

,A,B
2,2,b
3,3,b
4,4,b
5,5,b


In [101]:
dff.groupby("B").filter(lambda x: len(x) > 2, dropna=False)

,A,B
0,NaN,NaN
1,NaN,NaN
2,2.0,b
3,3.0,b
4,4.0,b
5,5.0,b
6,NaN,NaN
7,NaN,NaN


In [102]:
dff["C"] = np.arange(8)
dff.groupby("B").filter(lambda x: len(x["C"]) > 2)

,A,B,C
2,2,b,2
3,3,b,3
4,4,b,4
5,5,b,5


## Dispatching to instance methods
---

In [103]:
grouped = df.groupby("A")
grouped.agg(lambda x: x.std())

,C,D
A,,
bar,0.893219,1.100215
foo,0.870756,1.014376


In [104]:
grouped.std()

,C,D
A,,
bar,0.893219,1.100215
foo,0.870756,1.014376


In [105]:
tsdf = pd.DataFrame(
        np.random.randn(1000, 3),
        index=pd.date_range("1/1/2000", periods=1000),
        columns=["A", "B", "C"],
    )
tsdf.iloc[::2] = np.nan
tsdf

,A,B,C
2000-01-01,NaN,NaN,NaN
2000-01-02,-0.543611,-0.114113,0.452646
2000-01-03,NaN,NaN,NaN
2000-01-04,0.509926,0.429070,-1.467484
2000-01-05,NaN,NaN,NaN
...,...,...,...
2002-09-22,1.451068,-0.516541,-0.312509
2002-09-23,NaN,NaN,NaN
2002-09-24,-1.105090,-1.291940,0.219202
2002-09-25,NaN,NaN,NaN


In [108]:
grouped = tsdf.groupby(lambda x: x.year)
grouped.fillna(method="pad")

,A,B,C
2000-01-01,NaN,NaN,NaN
2000-01-02,-0.543611,-0.114113,0.452646
2000-01-03,-0.543611,-0.114113,0.452646
2000-01-04,0.509926,0.429070,-1.467484
2000-01-05,0.509926,0.429070,-1.467484
...,...,...,...
2002-09-22,1.451068,-0.516541,-0.312509
2002-09-23,1.451068,-0.516541,-0.312509
2002-09-24,-1.105090,-1.291940,0.219202
2002-09-25,-1.105090,-1.291940,0.219202


## Flexible apply
---

In [109]:
df

,A,B,C,D
0,foo,one,-1.610649,0.088701
1,bar,one,1.459099,-0.699461
2,foo,two,-0.377928,0.578508
3,bar,three,-0.324777,0.791631
4,foo,two,0.375390,0.707167
5,bar,two,0.649998,-1.355308
6,foo,one,-0.777828,-1.571484
7,foo,three,0.508563,-1.057590


In [110]:
grouped = df.groupby("A")

In [111]:
grouped["C"].apply(lambda x: x.describe())

A         
bar  count    3.000000
     mean     0.594773
     std      0.893219
     min     -0.324777
     25%      0.162611
     50%      0.649998
     75%      1.054548
     max      1.459099
foo  count    5.000000
     mean    -0.376490
     std      0.870756
     min     -1.610649
     25%     -0.777828
     50%     -0.377928
     75%      0.375390
     max      0.508563
Name: C, dtype: float64

In [112]:
grouped = df.groupby('A')['C']
def f(group):
        return pd.DataFrame({'original': group,
                             'demeaned': group - group.mean()})
grouped.apply(f)

,original,demeaned
0,-1.610649,-1.234159
1,1.459099,0.864325
2,-0.377928,-0.001438
3,-0.324777,-0.919550
4,0.375390,0.751880
5,0.649998,0.055225
6,-0.777828,-0.401337
7,0.508563,0.885054


In [113]:
def f(x):
    return pd.Series([x, x ** 2], index=["x", "x^2"])
s = pd.Series(np.random.rand(5))
s

0    0.321671
1    0.478189
2    0.146466
3    0.802762
4    0.313848
dtype: float64

In [114]:
s.apply(f)

,x,x^2
0,0.321671,0.103472
1,0.478189,0.228664
2,0.146466,0.021452
3,0.802762,0.644426
4,0.313848,0.098501


## Numba Accelerated Routines
---